In [2]:
import numpy as np

In [7]:

def init(m, n):
    return np.random.randint(2, size=(m,n))

def func(x):
    return -np.abs(np.sin(x) - 0.5 * x)

def decode(ss, a, b):
    n = ss.shape[1]
    x = []
    for s in ss:
        bin_to_int = np.array([int(j) << i for i,j in enumerate(s[::-1])]).sum()
        int_to_x = a + bin_to_int * (b - a) / (2**n - 1)
        x.append(int_to_x)
    return np.array(x)

def selection(pop, sample_size, fitness):
    m,n = pop.shape
    print('=====INSIDE SELECTION=====')
    print(f'pop shape is: {pop.shape}')
    new_pop = pop.copy()
        
    for i in range(m):
        rand_id = np.random.choice(m, size=max(1, int(sample_size*m)), replace=False)
        max_id = rand_id[fitness[rand_id].argmax()]
        new_pop[i] = pop[max_id].copy()
        print(f'At {i} new_pop is: {new_pop}')
    print('=====END SELECTION=====')
    return new_pop

def crossover(pop, pc):
    m,n = pop.shape
    new_pop = pop.copy()
    print('=====INSIDE CROSSOVER=====')
    print(f'pop shape is: {pop.shape}')
    for i in range(0, m-1, 2):
        if np.random.uniform(0, 1) < pc:
            pos = np.random.randint(0, n-1)
            new_pop[i, pos+1:] = pop[i+1, pos+1:].copy()
            new_pop[i+1, pos+1:] = pop[i, pos+1:].copy()
    print('=====END CROSSOVER=====')        
    return new_pop

def mutation(pop, pm):
    m,n = pop.shape
    new_pop = pop.copy()
    print('=====INSIDE MUTATION=====')
    print(f'pop shape is: {pop.shape}')
    mutation_prob = (np.random.uniform(0, 1, size=(m,n)) < pm).astype(int)
    print(f'mutation_prob is: {mutation_prob}')
    print(f'mutation_prob + new_pop is: {(mutation_prob + new_pop) % 2}')
    print('=====END MUTATION=====')
    return (mutation_prob + new_pop) % 2

def print_result(gen_num, pop, fitness, x):
    m = pop.shape[0]
    print('=' * 68)
    print(f'Generation {gen_num} max fitness {fitness.max():0.4f} at x = {x[fitness.argmax()]:0.4f}')
    
    for i in range(m):
        print(f'# {i+1}\t{pop[i]}   fitness: {fitness[i]:0.4f}')
    
    print(f'Average fitness: {fitness.mean():0.4f}')
    print('=' * 68, '\n')

def GeneticAlgorithm(func, pop_size, str_size, low, high, 
                     ps=0.2, pc=1.0, pm=0.1, max_iter=10, eps=1e-5, random_state=None):
    
    np.random.seed(random_state)
    pop = init(pop_size, str_size)
    print(f'Pop after _init_ is: {pop} and shape is: {pop.shape}')
    x = decode(pop, low, high)
    print(f'Pop after decode is: {x} and shape is: {x.shape}')
    fitness = func(x)
    print(f'fitness after func is: {fitness} and shape is: {fitness.shape}')
    best = [fitness.max()]
    #print_result(1, pop, fitness, x)
    
    i = 0
    while i < max_iter and abs(best[-1]) > eps:
        pop = selection(pop, ps, fitness)
        print(f'Iter: {i}, Population after selection is: {pop}')
        pop = crossover(pop, pc)
        print(f'Iter: {i}, Population after crossover is: {pop}')
        pop = mutation(pop, pm)
        print(f'Iter: {i}, Population after mutation is: {pop}')
        x = decode(pop, low, high)
        fitness = func(x)
        best.append(fitness.max())
        i += 1
    
    print_result(i, pop, fitness, x)
    
    if i == max_iter:
        print(i, 'maximum iteration reached!')
        print('Solution not found. Try increasing max_iter for better result.')
    else:
        print('Solution found at iteration', i)
        
    return fitness, x, best, i, pop_size    

In [8]:
fs, xs, best, i, m = GeneticAlgorithm(func, pop_size=4, str_size=4, low=1, high=3, random_state=69)
print(f'fs: {fs}, xs: {xs}, i:{i}, m:{m}')

Pop after _init_ is: [[0 1 1 1]
 [0 1 0 1]
 [0 1 0 1]
 [0 0 0 0]] and shape is: (4, 4)
Pop after decode is: [1.93333333 1.66666667 1.66666667 1.        ] and shape is: (4,)
fitness after func is: [-0.03166658 -0.16207462 -0.16207462 -0.34147098] and shape is: (4,)
=====INSIDE SELECTION=====
pop shape is: (4, 4)
At 0 new_pop is: [[0 0 0 0]
 [0 1 0 1]
 [0 1 0 1]
 [0 0 0 0]]
At 1 new_pop is: [[0 0 0 0]
 [0 1 1 1]
 [0 1 0 1]
 [0 0 0 0]]
At 2 new_pop is: [[0 0 0 0]
 [0 1 1 1]
 [0 1 0 1]
 [0 0 0 0]]
At 3 new_pop is: [[0 0 0 0]
 [0 1 1 1]
 [0 1 0 1]
 [0 1 0 1]]
=====END SELECTION=====
Iter: 0, Population after selection is: [[0 0 0 0]
 [0 1 1 1]
 [0 1 0 1]
 [0 1 0 1]]
=====INSIDE CROSSOVER=====
pop shape is: (4, 4)
=====END CROSSOVER=====
Iter: 0, Population after crossover is: [[0 0 0 1]
 [0 1 1 0]
 [0 1 0 1]
 [0 1 0 1]]
=====INSIDE MUTATION=====
pop shape is: (4, 4)
mutation_prob is: [[0 0 0 1]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 0 0]]
mutation_prob + new_pop is: [[0 0 0 0]
 [0 1 0 0]
 [1 1 0 1]
 [